In [ ]:
# First of all, let's set the authentication info for
# API Highways.
api_base = "https://api.apihighways.org/v1"
credentials = input()
api_headers = {
    "Content-Type": "application/json",
    "User-Agent": "Python3.6requests",
    "Authorization": f"Bearer {credentials}"
}

# 👇 Remove the output below! DON'T COMMIT!

In [131]:
# We are interested on a night lights map. Night lights can provide
# a superb window into patterns of human settlement.

# We'll build some scaffolding so we can check the API more easily:
import requests
import json

def get_from_api(endpoint, attr_dict, api_base = api_base, api_headers = api_headers):
    attrs = '&'.join([f"{k}={v}" for k, v in attr_dict.items()])
    attrs_str = '?' + attrs if attrs is not "" else ""
    final_endpoint = f"{api_base}{endpoint}{attrs_str}"
    print(f"Making request to {final_endpoint}")
    # print(api_headers)
    result = json.loads(
        requests.get(
            final_endpoint,
            headers = api_headers
        ).text
    )
    try:
        return result['data']
    except KeyError: # No ['data']: no data
        print(result)
        return []

In [95]:
# To get the basic, paginated index
# get_from_api('/dataset', {})

# All GEE datasets
# get_from_api('/dataset', {"page[size]": 1000, "provider": "gee"})

# Let's look for the DMSP OLS Night Lights data
dmsp_dataset = get_from_api('/dataset', {
    "page[size]": 1000,
    "provider": "gee",
    "name": "dmsp",         # We want to search for 'dmsp'
    "includes": "metadata"  # And include metadata
})

dmsp_dataset

Making request to https://api.apihighways.org/v1/dataset?page[size]=1000&provider=gee&name=dmsp&includes=metadata


[{'id': 'c1c7dc51-9550-4b3b-b9e7-62b0630eb7af',
  'type': 'dataset',
  'attributes': {'name': 'DMSP OLS: Global Radiance-Calibrated Nighttime Lights Version 4, Defense Meteorological Program Operational Linescan System',
   'slug': 'DMSP-OLS-Global-Radiance-Calibrated-Nighttime-Lights-Version-4-Defense-Meteorological-Program-Operational-Linescan-System',
   'type': None,
   'dataPath': None,
   'attributesPath': None,
   'connectorType': 'rest',
   'provider': 'gee',
   'userId': '5aad5c86a61d3ddd586e5861',
   'connectorUrl': None,
   'tableName': 'NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4',
   'status': 'saved',
   'published': True,
   'sandbox': False,
   'overwrite': False,
   'verified': False,
   'blockchain': {},
   'subscribable': {},
   'env': 'production',
   'geoInfo': False,
   'protected': False,
   'legend': {'date': [], 'region': [], 'country': [], 'nested': []},
   'clonedHost': {},
   'errorMessage': '',
   'taskId': None,
   'createdAt': '2018-06-08T14:15:53.577Z',
   'updat

In [132]:
# We know what dataset to use, so let's
# move to GEE for the rest of the analysis
import ee
# Earth Engine is managed by Google and currently
# by invitation only. You'll have to apply to use
# the service and authorize the library prior to
# using it. Instructions for doing so can be found
# at https://developers.google.com/earth-engine/python_install
ee.Initialize()

# It might not work at first. In that case, run the
# code below. Notice it's not python code but shell
# commands. Your Google account needs access for it.
# As always: take care of removing the token before
# commiting any code

In [ ]:
%%bash
earthengine authenticate --quiet

In [ ]:
%%bash
earthengine authenticate --authorization-code=<your-code>

In [133]:
# We'll use data from the Defense Meteorological Satellite
# Program, provided by the US Department of Defense. DMSP 
# provides the Operational Linescan System, which is used to
# monitor clouds. The instrument is also used to capture
# nighttime imagery. Human settlements emit light at night,
# so this kind of information can be used to track trends in
# habitation patterns.

# First of all, we'll take the data from GEE into our notebook
collection = ee.ImageCollection(
              # 👇 NOAA/DMSP-OLS/CALIBRATED_LIGHTS_V4
    dmsp_dataset[0]['attributes']['tableName']
)
# Notice that the calibrated lights dataset is an ImageCollection

# Next step is to add a band (with the same value in its whole
# extension) so we can work with it and a reducer. This value
# is going to be the date the image was taken in an easier format.
# to consume.
# The usual step is to create a function that can consume an 
# Image, and later map it through the ImageCollection
def create_time_band(img):
    year = img.date().difference(ee.Date('1990-01-01'), 'year')
    return ee.Image(year).float().addBands(img)

collection_with_date = collection.select(
    'avg_vis'
).map(
    create_time_band
)

# the linearFit reducer expects two bands, the first with the
# independent variable and another one with the dependent one
fit = collection_with_date.reduce(ee.Reducer.linearFit());
# The result of reducing an ImageCollection is an Image
fit.getInfo()

{'type': 'Image',
 'bands': [{'id': 'scale',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]},
  {'id': 'offset',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}

In [134]:
# We can visualize the results. The easiest way is to generate a thumbUrl
single_image_thumburl = ee.Image(collection.first()).getThumbURL(
    {
        "bands": 'avg_vis',
        "min": 0,
        "max": 63,
        "dimensions": [1440, 720],
        "palette": "black,white"
    }
)

from IPython import display
display.Image(url = single_image_thumburl)

In [135]:
# Or we can show an interactive map of the trend
import folium

trends_mapid = fit.getMapId(
    {
        "bands": 'scale,offset,scale',
        "min": '0,0,0',
        "max": '0.18,0.20,-0.18',
    }
)
# The rationale of this visualization is to visualize
# the base brightness (the offset) in green, and the trend
# into the red/blue axis.

# GEE can generate slippy maps from `Images`
trends_mapid_endpoint = f"https://earthengine.googleapis.com/map/{trends_mapid['mapid']}/{{z}}/{{x}}/{{y}}?token={trends_mapid['token']}"
fit_map = folium.Map(
    location=[0, 0],
    zoom_start = 2,
    tiles = trends_mapid_endpoint,
    attr='API Highways'
)

fit_map

In [136]:
# We'll look in the API for an administrative divisions dataset:
admin_div_datasets = get_from_api('/dataset', {
    "provider": "gee",
    "name": "boundary",         # We want to search for 'dmsp'
    "includes": "metadata"        # And include metadata
}, api_headers = api_headers)
admin_div_datasets

Making request to https://api.apihighways.org/v1/dataset?provider=gee&name=boundary&includes=metadata


[{'id': '56532421-2e1c-4021-8743-ded130319175',
  'type': 'dataset',
  'attributes': {'name': 'LSIB: Large Scale International Boundary Polygons, Detailed',
   'slug': 'LSIB-Large-Scale-International-Boundary-Polygons-Detailed',
   'type': None,
   'dataPath': None,
   'attributesPath': None,
   'connectorType': 'rest',
   'provider': 'gee',
   'userId': '5aad5c86a61d3ddd586e5861',
   'connectorUrl': None,
   'tableName': 'USDOS/LSIB/2013',
   'status': 'saved',
   'published': True,
   'sandbox': False,
   'overwrite': False,
   'verified': False,
   'blockchain': {},
   'subscribable': {},
   'env': 'production',
   'geoInfo': False,
   'protected': False,
   'legend': {'date': [], 'region': [], 'country': [], 'nested': []},
   'clonedHost': {},
   'errorMessage': '',
   'taskId': None,
   'createdAt': '2018-06-08T14:23:58.758Z',
   'updatedAt': '2018-06-08T14:23:59.368Z',
   'metadata': [{'id': '5b1a918503ebe20011651e39',
     'type': 'metadata',
     'attributes': {'dataset': '5653

In [127]:
# The first one looks enough for our purposes
admin_div_dataset_id, admin_div_dataset_tablename  = admin_div_datasets[0]['id'], admin_div_datasets[0]['attributes']['tableName']
(admin_div_dataset_id, admin_div_dataset_tablename)

('56532421-2e1c-4021-8743-ded130319175', 'USDOS/LSIB/2013')

In [138]:
# GEE FeatureCollections are exposed in the API Highways query endpoint
# -- the features' attributes are exposed as a table with a sql-like
# interface. In this case we are interested in the first feature with
# a 'GHA' iso_alpha3. Notice that the response is always a list of feature
# attributes. The 'system:index' field will then allow the request of the
# object from GEE.

ghana_ee_system_index = get_from_api('/query', {
   "sql": f"select system:index from 56532421-2e1c-4021-8743-ded130319175 where iso_alpha3 = GHA limit 1"
}, api_headers = api_headers)[0]
ghana_ee_system_index

Making request to https://api.apihighways.org/v1/query?sql=select system:index from 56532421-2e1c-4021-8743-ded130319175 where iso_alpha3 = GHA limit 1


{'system:index': '0000355638593a0e8f15'}

In [140]:
# We can request a GEE object from the tableName
boundaries_layer = ee.FeatureCollection(admin_div_dataset_tablename)
# All features in GEE have a 'system:index'
ghana_feature = boundaries_layer.filter(ee.Filter.eq("system:index", ghana_ee_system_index['system:index']))

# It's possible to filter geometries too directly from GEE
sierra_leone_feature = boundaries_layer.filter(ee.Filter.eq('iso_alpha3', 'SLE'))
# We are interested in the first element of the features dict, that's were
# the geometry resides
ghana_dict = ghana_feature.getInfo()['features'][0]
# the feature is in the GeoJSON format
# so this would work for exporting the
# feature and inspecting it in a GIS
# package
with open('ghana.json', 'w') as file:
    json.dump(ghana_dict, file)
    
    
# That json can be shown on a map
ghana_geojson = os.path.join('.', 'ghana.json')

m = folium.Map(location=[8, -1], zoom_start=6)

folium.GeoJson(
    ghana_geojson,
    name='geojson'
).add_to(m)

folium.TileLayer(
    trends_mapid_endpoint,
    attr='API Highways'
).add_to(m)

folium.LayerControl().add_to(m)

m